<a href="https://colab.research.google.com/github/Cris-0506/Cris-0506/blob/main/Enem_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Projeto Enem 2021 / Dados disponíveis no site do Inep (http://inep.gov.br/microdados).


Análise de desempenho na nota de redação entre homens e mulheres na prova do Enem 2021.

In [9]:
import pandas as pd
import numpy as np

In [10]:
dados = pd.read_csv('/content/MICRODADOS_ENEM_2021.csv', sep=';', encoding = 'iso-8859-1')

In [11]:
np.random.seed(1)

In [12]:
amostragem = dados.sample(30000)

In [13]:
amostragem.shape

(30000, 76)

In [ ]:
# Analisando as variáveis do dataframe.
amostragem.columns.values

In [15]:
# Filtrando as colunas que serão utilizadas.
seleçao = ['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO','TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE','TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO','IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC','CO_UF_ESC','TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4', 'NU_NOTA_COMP5','NU_NOTA_REDACAO']
       

In [16]:
dados_enem = amostragem.filter(items=seleçao)

In [ ]:
dados_enem.head(30)

In [18]:
dados_sexo = dados_enem ['TP_SEXO']

In [ ]:
dados_sexo.value_counts()

In [20]:
# Renomeando registros da faixa etária.
dados_enem['TP_FAIXA_ETARIA'] = dados_enem['TP_FAIXA_ETARIA'].replace({1:'Menor de 17', 2:17, 3:18, 4:19, 5:20, 6:21,7:22, 8:23, 9:24, 10:25, 11:'Entre 26 e 30', 12: 'Entre 31 e 35', 13: 'Entre 36 e 40', 14: 'Entre 41 e 45', 15: 'Entre 46 e 50', 16: 'Entre 51 e 55', 17:'Entre 56 e 60', 18: 'Entre 61 e 65', 19: 'Entre 66 e 70', 20: 'Maior de 70' })

In [21]:
dados_faixa_etaria = dados_enem ['TP_FAIXA_ETARIA']

In [ ]:
dados_faixa_etaria.value_counts()

In [23]:
dados_enem['TP_ESTADO_CIVIL'] = dados_enem['TP_ESTADO_CIVIL']. replace ({0: 'Não informado', 1: 'Solteiro(a)', 2: 'Casado(a)', 3: 'Divorciado(a)/Separado(a)', 4: 'Viúvo(a)'})

In [24]:
# Criando uma variável que contém somente os candidatos que compareceram a prova e não foram desclassificados.
candidatos = dados_enem.loc[dados_enem.TP_PRESENCA_CH == 1]

In [25]:
# A análise será feita utilizando somente os dados dos vestibulandos classificados. As provas LC, CH e redação foram feitas no mesmo dia, CN e MT em outro dia.
# Assim, o aluno perde mais de uma prova caso falte um dia.
# Registros: 0 = Faltou  à prova; 1 = Presente na prova; 2 = Eliminado na prova.



In [26]:
candidatos = candidatos.loc[candidatos.TP_PRESENCA_MT == 1]

In [27]:
dados_estado_civil = candidatos['TP_ESTADO_CIVIL']

In [ ]:
dados_estado_civil.value_counts()

In [29]:
dados_mulheres = candidatos.loc[candidatos.TP_SEXO == 'F']

In [ ]:
dados_mulheres.shape

In [31]:
dados_homens = candidatos[candidatos.TP_SEXO == 'M']

In [ ]:
dados_homens.shape

In [33]:
# Retirando amostras de 3000 homens e 3000 mulheres para melhorar a robustez do teste de normalidade.
amostra_mulheres = dados_mulheres.sample(3000)

In [34]:
amostra_homens = dados_homens.sample(3000)

1. Objetivo: Analisar se há diferença de desempenho entre homens e mulheres na nota de redação.


*   H0 = Não existe diferença (p>0,05).
*   Ha = Existe diferença (p<=0,05).



In [35]:
import plotly.express as px
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

In [36]:
# Analisando Normalidade com teste de Shapiro-Wilk. (p>0,05)
stats.shapiro(amostra_mulheres.NU_NOTA_REDACAO)

ShapiroResult(statistic=0.9464705586433411, pvalue=1.0187863777277264e-31)

In [37]:
stats.shapiro(amostra_homens.NU_NOTA_REDACAO)

ShapiroResult(statistic=0.9393039345741272, pvalue=2.145908651122257e-33)

In [38]:
# A distribuição das notas de redação de homens e mulheres não é normal, dessa forma será necessário um teste não paramétrico.
# Observação dos dados nos gráficos QQ Plot e histograma.

In [ ]:
stats.probplot(amostra_mulheres['NU_NOTA_REDACAO'], dist='norm', plot=plt)
plt.title('Normal QQ Plot-Nota Redação Mulheres')
plt.show()

In [ ]:
stats.probplot(amostra_homens['NU_NOTA_REDACAO'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Nota Redação Homens')
plt.show()

In [ ]:
sns.histplot(amostra_mulheres, x='NU_NOTA_REDACAO', bins=10, color='orange', kde=True, stat='probability')

In [ ]:
sns.histplot(amostra_homens, x='NU_NOTA_REDACAO', bins=10, color= 'red', kde=True, stat='probability')

In [43]:
stat, p=stats.mannwhitneyu(amostra_mulheres.NU_NOTA_REDACAO, amostra_homens.NU_NOTA_REDACAO)
print('Estatística do Teste: {}'.format(stat.round(2)))
print('p-valor:{}'.format(p))

Estatística do Teste: 4796493.5
p-valor:9.754372085408898e-06


In [ ]:
amostra_mulheres.describe()

In [ ]:
amostra_homens.describe()

Conclusão: Considerando o intervalo de confiança de 95%, podemos concluir que existe diferença entre as medianas das notas de redação entre homens e mulheres. Rejeita H0. O desempenho das mulheres na prova de redação foi superior ao dos homens.


*   Mediana nota mulheres = 620
*   Mediana nota homens = 600



2. Objetivo: Analisar se há diferença de desempenho entre mulheres casadas e homens casados na nota de redação.


*   H0 = Não há diferença (p>0,05)
*   Ha = Há diferença (p<=0,05)


In [46]:
mulheres_casadas = amostra_mulheres.loc[amostra_mulheres.TP_ESTADO_CIVIL == 'Casado(a)']

In [47]:
mulheres_casadas.shape

(65, 29)

In [48]:
homens_casados = amostra_homens.loc[amostra_homens.TP_ESTADO_CIVIL == 'Casado(a)']

In [49]:
homens_casados.shape

(116, 29)

In [50]:
stats.shapiro(mulheres_casadas.NU_NOTA_REDACAO)

ShapiroResult(statistic=0.9224055409431458, pvalue=0.0005648444057442248)

In [51]:
stats.shapiro(homens_casados.NU_NOTA_REDACAO)

ShapiroResult(statistic=0.8949728608131409, pvalue=1.619147980136404e-07)

In [52]:
# A distribuição das notas de redação de homens e mulheres casados não é normal, dessa forma será necessário um teste não paramétrico.
# Observação dos dados nos gráficos QQ Plot e histograma.

In [ ]:
stats.probplot(mulheres_casadas ['NU_NOTA_REDACAO'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Nota Redação Mulheres Casadas')
plt.show()

In [ ]:
stats.probplot(homens_casados['NU_NOTA_REDACAO'], dist='norm', plot=plt)
plt.title('Normal QQ Plot - Nota Redação Homens Casados')
plt.show()

In [ ]:
sns.histplot(mulheres_casadas, x='NU_NOTA_REDACAO', bins=10, color='orange', kde=True, stat='probability')

In [ ]:
sns.histplot(homens_casados, x='NU_NOTA_REDACAO', bins=10, color='blue', kde=True, stat='probability')

In [57]:
stat, p = stats.mannwhitneyu(mulheres_casadas.NU_NOTA_REDACAO, homens_casados.NU_NOTA_REDACAO)
print('Estatística do Teste: {}:'.format(stat.round(2)))
print('p-valor:{}'.format(p))

Estatística do Teste: 3558.5:
p-valor:0.5318539961770496


Conclusão: Considerando o intervalo de confiança de 95%, não podemos concluir que exista diferença entre as medianas das notas de redação entre mulheres casadas e homens casados. Não rejeita H0.